# network diagram


In [1]:
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import *

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import combinations
from scipy import stats


plt.style.use('../style.mplstyle')
df = pd.read_csv("../latest.csv")

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D

c:\Users\sevi\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
df = fix_data(df, drop_zeros=False)

In [3]:
print("length of df: ", len(df))
print('\n')
df.head(1)

length of df:  106




,StartDate,reading,Duration,Progress,Finished,Q42,Q43,Q1,Q1_7_TEXT,Q2,...,Q16_3,Q16_4,Q16_5,Q16_6,Q16_7,Q16_8,Q16_9,Q17,Q18,Q46
0,2024-08-08 23:30:29,7,118,100,True,Agree,LinkedIn DM,Industry,NaN,2.0,...,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN


In [4]:
df.columns

Index(['StartDate', 'reading', 'Duration', 'Progress', 'Finished', 'Q42',
       'Q43', 'Q1', 'Q1_7_TEXT', 'Q2', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q4_1',
       'Q4_2', 'Q4_3', 'Q4_4', 'Q5', 'Q6_1', 'Q6_2', 'Q6_3', 'Q6_4', 'Q6_5',
       'Q6_6', 'Q6_7', 'Q6_8', 'Q6_9', 'Q15', 'Q16_1', 'Q16_2', 'Q16_3',
       'Q16_4', 'Q16_5', 'Q16_6', 'Q16_7', 'Q16_8', 'Q16_9', 'Q17', 'Q18',
       'Q46'],
      dtype='object')

# WANT the scores for timelines, the scores for each QUESTION, familiarity with each alignment concept, familiarity with empirical ML, when should we build?

In [5]:
# Q3: How familiar are you with the following empirical AI concepts?
# Q4: How familiar are you with the following theoretical AI Alignment concepts?
# Q5: Which best describes your position on when we should build AGI (Artificial General Intelligence)? # For the purposes of this survey, we define AGI as: AI systems that are better at STEM research than the best human scientists, in addition to potentially having other advanced capabilities.

# Q6: rate your agreement with the following statements

# replace the qualitative data with the quantitative data

In [6]:
df['Q3_1'].value_counts().sort_index()

Know a fair amount    23
Know a little         13
Know it well          50
Name: Q3_1, dtype: int64

In [7]:
timelines_positions = {
    "We should develop more powerful and more general systems as fast as possible": 5,
    "Soon, but not as fast as possible": 3.75,
    "Eventually, but not soon.": 2.5,
    "We should never build AGI": 1.25
}

agreement_swaps = {
    "strongly disagree": 1,
    "Somewhat disagree": 2,
    "Neither agree nor disagree": 3,
    "Somewhat agree": 4,
    "Strongly agree": 5,
}

familiarity_swaps = {
    "Never heard of it": 1,
    "Heard of it": 2,
    "Know a little": 3,
    "Know a fair amount": 4,
    "Know it well": 5,
}

pre_questions = [f'Q6_{i}' for i in range(1, 10)]
familiarity_questions = [f'Q3_{i}' for i in range(1, 4)]
alignment_familiarity_questions = [f'Q4_{i}' for i in range(1, 5)]
timelines_questions = "Q5"


In [8]:
# get just the df of the questions we care about
df = df[pre_questions + familiarity_questions + alignment_familiarity_questions +  [timelines_questions]]
# df = df[pre_questions + [timelines_questions]]

In [9]:
# replace the strings with numbers
df = df.replace({
    **agreement_swaps,
    **familiarity_swaps,
    **timelines_positions,
})

df

,Q6_1,Q6_2,Q6_3,Q6_4,Q6_5,Q6_6,Q6_7,Q6_8,Q6_9,Q3_1,Q3_2,Q3_3,Q4_1,Q4_2,Q4_3,Q4_4,Q5
0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,1.0,1.0,1.0,1.0,1.0,5.00
1,4.0,4.0,1.0,3.0,4.0,2.0,3.0,4.0,5.0,4.0,4.0,2.0,5.0,1.0,1.0,1.0,3.75
2,4.0,1.0,1.0,3.0,3.0,4.0,1.0,3.0,4.0,5.0,5.0,5.0,3.0,3.0,1.0,1.0,2.50
3,2.0,4.0,2.0,3.0,4.0,2.0,4.0,1.0,5.0,5.0,4.0,1.0,1.0,2.0,3.0,1.0,1.25
4,4.0,4.0,2.0,3.0,2.0,1.0,3.0,2.0,5.0,5.0,5.0,2.0,1.0,1.0,1.0,1.0,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,4.0,4.0,2.0,5.0,3.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,2.0,1.0,1.0,1.0,2.50
103,4.0,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,5.0,4.0,1.0,3.0,2.0,1.0,1.0,5.00
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# rename columns Q6_1 -> Q6_9 to "agreement_x"
df = df.rename(columns={k: f"agreement_{i}" for i, k in enumerate(pre_questions, 1)})

# rename columns Q3_1 -> Q3_3 to "familiarity_x"
df = df.rename(columns={k: f"familiarity_{i}" for i, k in enumerate(familiarity_questions, 1)})

# rename columns Q4_1 -> Q4_4 to "alignment_x"
df = df.rename(columns={k: f"alignment_{i}" for i, k in enumerate(alignment_familiarity_questions, 1)})

# rename Q5 to "timelines"
df = df.rename(columns={timelines_questions: "timelines"})

In [11]:
df.columns

Index(['agreement_1', 'agreement_2', 'agreement_3', 'agreement_4',
       'agreement_5', 'agreement_6', 'agreement_7', 'agreement_8',
       'agreement_9', 'familiarity_1', 'familiarity_2', 'familiarity_3',
       'alignment_1', 'alignment_2', 'alignment_3', 'alignment_4',
       'timelines'],
      dtype='object')

In [12]:
# drop the rows with NaN
df = df.dropna()
df

,agreement_1,agreement_2,agreement_3,agreement_4,agreement_5,agreement_6,agreement_7,agreement_8,agreement_9,familiarity_1,familiarity_2,familiarity_3,alignment_1,alignment_2,alignment_3,alignment_4,timelines
0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,1.0,1.0,1.0,1.0,1.0,5.00
1,4.0,4.0,1.0,3.0,4.0,2.0,3.0,4.0,5.0,4.0,4.0,2.0,5.0,1.0,1.0,1.0,3.75
2,4.0,1.0,1.0,3.0,3.0,4.0,1.0,3.0,4.0,5.0,5.0,5.0,3.0,3.0,1.0,1.0,2.50
3,2.0,4.0,2.0,3.0,4.0,2.0,4.0,1.0,5.0,5.0,4.0,1.0,1.0,2.0,3.0,1.0,1.25
4,4.0,4.0,2.0,3.0,2.0,1.0,3.0,2.0,5.0,5.0,5.0,2.0,1.0,1.0,1.0,1.0,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,3.0,4.0,4.0,3.0,3.0,4.0,3.0,2.0,3.0,5.0,4.0,1.0,1.0,1.0,1.0,1.0,3.75
100,5.0,3.0,1.0,3.0,1.0,1.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,1.25
102,4.0,4.0,2.0,5.0,3.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,2.0,1.0,1.0,1.0,2.50
103,4.0,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,5.0,4.0,1.0,3.0,2.0,1.0,1.0,5.00


In [23]:
# take the df of ONLY people who have 4 or 5 on their alignment_3 question


In [33]:
# calculate the percent of df_safety who answered 4 or 5 on agreement_5
def percent_four_or_five(question, familiarity):

    df_safety = df[df[familiarity] >= 4]
    df_other = df[df[familiarity] < 4]
    percent_safety = len(df_safety[df_safety[question] >= 4]) / len(df_safety)

    print(f"SAFETY PPL AGREE TO: {question} AT A RATE OF: {percent_safety}")

    percent_other = len(df_other[df_other[question] >= 4]) / len(df_other)
    print(f"percent of other people who AGREE on {question}: {percent_other}")

In [34]:
for i in range(1, 10):
    percent_four_or_five(f"agreement_{i}", "alignment_2")

SAFETY PPL AGREE TO: agreement_1 AT A RATE OF: 0.2631578947368421
percent of other people who AGREE on agreement_1: 0.46774193548387094
SAFETY PPL AGREE TO: agreement_2 AT A RATE OF: 0.631578947368421
percent of other people who AGREE on agreement_2: 0.5645161290322581
SAFETY PPL AGREE TO: agreement_3 AT A RATE OF: 0.42105263157894735
percent of other people who AGREE on agreement_3: 0.20967741935483872
SAFETY PPL AGREE TO: agreement_4 AT A RATE OF: 0.3157894736842105
percent of other people who AGREE on agreement_4: 0.4032258064516129
SAFETY PPL AGREE TO: agreement_5 AT A RATE OF: 0.3684210526315789
percent of other people who AGREE on agreement_5: 0.46774193548387094
SAFETY PPL AGREE TO: agreement_6 AT A RATE OF: 0.15789473684210525
percent of other people who AGREE on agreement_6: 0.4032258064516129
SAFETY PPL AGREE TO: agreement_7 AT A RATE OF: 0.7894736842105263
percent of other people who AGREE on agreement_7: 0.43548387096774194
SAFETY PPL AGREE TO: agreement_8 AT A RATE OF: 0.1

In [35]:
for i in range(1, 10):
    percent_four_or_five(f"agreement_{i}", "alignment_3")

SAFETY PPL AGREE TO: agreement_1 AT A RATE OF: 0.4117647058823529
percent of other people who AGREE on agreement_1: 0.421875
SAFETY PPL AGREE TO: agreement_2 AT A RATE OF: 0.5882352941176471
percent of other people who AGREE on agreement_2: 0.578125
SAFETY PPL AGREE TO: agreement_3 AT A RATE OF: 0.29411764705882354
percent of other people who AGREE on agreement_3: 0.25
SAFETY PPL AGREE TO: agreement_4 AT A RATE OF: 0.29411764705882354
percent of other people who AGREE on agreement_4: 0.40625
SAFETY PPL AGREE TO: agreement_5 AT A RATE OF: 0.4117647058823529
percent of other people who AGREE on agreement_5: 0.453125
SAFETY PPL AGREE TO: agreement_6 AT A RATE OF: 0.17647058823529413
percent of other people who AGREE on agreement_6: 0.390625
SAFETY PPL AGREE TO: agreement_7 AT A RATE OF: 0.5882352941176471
percent of other people who AGREE on agreement_7: 0.5
SAFETY PPL AGREE TO: agreement_8 AT A RATE OF: 0.23529411764705882
percent of other people who AGREE on agreement_8: 0.171875
SAFETY